In [1]:
from langchain_groq import ChatGroq

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
llm = ChatGroq(
    temperature=0,
    groq_api_key = os.getenv('GROQ_API_KEY'),
    model_name="llama-3.1-70b-versatile"  
)
response = llm.invoke("who are you?")
print(response.content)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.talentport.com/job-description/detail/back-end-developer-job-description")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Back-end Developer Job Description - Talentport










Our TalentsSoftware EngineersTech LeadBack End DevelopersFront End DevelopersDevops EngineersSDET EngineersAI EngineersSalesSDRsCustomer SuccessDigital MarketersSEO ExpertsSocial Media MarketersPerformance MarketersContent WritersFractional CMOCreative TalentsGraphic DesignersVideo EditorsBlogApply as a TalentBack-end Developer Job DescriptionA customizable job description template for hiring world-class back-end developers. Use this as the starting point for your next hire.Hire a TalentCATEGORIESDigital MarketingSoftware EngineeringWeb DevelopmentCreativeAdministrativeHire a Top Talent NowA back-end developer is a type of software developer who focuses on creating and maintaining the server side of web applications. They work on the behind-the-scenes functionality of websites or web applications, handling tasks such as database management, server configuration, and building the logic that enables the front end of the applicatio

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website. 
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: 'role', 'experience', 'skills', and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE and term json and signs ``` in structure):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)


str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Back-end Developer',
 'experience': 'Not specified',
 'skills': ['Basic understanding of front-end technologies and platforms, including JavaScript, HTML5, and CSS3',
  'Familiarity with server-side CSS preprocessors like LESS and SASS',
  'Knowledge of accessibility and security compliance (depending on the project)',
  'Experience with user authentication and authorization across multiple systems',
  'Integration of multiple data sources and databases',
  'Management of hosting environment, including database administration and scalability',
  'Data migration, transformation, and scripting',
  'Setting up and administering backups',
  'Generating data in various formats',
  'Adapting output for different delivery platforms (e.g., mobile vs. desktop)',
  'Creating database schemas that support business processes',
  'Implementing automated testing platforms and unit tests',
  'Proficiency in at least one back-end programming language (e.g., PHP, Python, Ruby, Java, .NET, Jav

In [31]:
type(json_res)

dict

In [25]:
import pandas as pd

df = pd.read_csv("app\\resource\\my_portfolio.csv")

df

,Techstack,Links
0,"Generative AI , Langchain, Pandas, Ollama 3.1",link GitHub1
1,"Full-stack HR Salary Project , C# , .ASAP ,PHP...",link GitHub2
2,"NLP, NLTK, SpaCy, Pandas, Numpy",link GitHub4
3,"React, Node.js, MongoDB",link GitHub5
4,"QA Automation Testing , Java, Selenium Web Dri...",link GitHub6


In [26]:
import chromadb
import uuid

client  = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
       collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [27]:
links = collection.query(query_texts=['skills'], n_results=2).get('metadatas', [])
links
                        

[[{'links': 'link GitHub3'}, {'links': 'link GitHub5'}]]

In [28]:
job 

''

In [29]:
job = json_res
job['skills']

['Basic understanding of front-end technologies and platforms, including JavaScript, HTML5, and CSS3',
 'Familiarity with server-side CSS preprocessors like LESS and SASS',
 'Knowledge of accessibility and security compliance (depending on the project)',
 'Experience with user authentication and authorization across multiple systems',
 'Integration of multiple data sources and databases',
 'Management of hosting environment, including database administration and scalability',
 'Data migration, transformation, and scripting',
 'Setting up and administering backups',
 'Generating data in various formats',
 'Adapting output for different delivery platforms (e.g., mobile vs. desktop)',
 'Creating database schemas that support business processes',
 'Implementing automated testing platforms and unit tests',
 'Proficiency in at least one back-end programming language (e.g., PHP, Python, Ruby, Java, .NET, JavaScript)',
 'Familiarity with code versioning tools like Git',
 'Understanding of OWAS

In [30]:
prompt_email = PromptTemplate.from_template(
"""
### JOB DESCRIPTION:
{job_description}

###

You are Florina, a first-year student at the Faculty of Automation and Computers, specializing in Informatics. You are highly motivated to deepen your understanding of programming, algorithms, and software development. 
You recently completed a master's degree in Applied Information Systems for Production and Services in the same Faculty, which gave you a strong foundation in practical applications of information systems.
As a student with both academic and practical experience, you are eager to apply your knowledge to real-world problems. Your goal is to analyze the problem, break it down into smaller steps, and use best practices in software development to complete the task. 
Additionally, reflect on how the theoretical knowledge from your studies can be applied in this scenario. 
Remember you are Florina, a student with master degree and with personal projects {link_list} and first-year student at the Faculty of Automation and Computers, specializing in Informatics
Do not provide a preamble.
### EMAIL (NO PREAMBLE): 

""" 
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Application for Back-end Developer Position

Dear Hiring Manager,

I am excited to apply for the Back-end Developer position at your company. With a strong foundation in information systems and a passion for programming, I am confident that I can make a valuable contribution to your team.

As a recent master's graduate in Applied Information Systems for Production and Services and a first-year student in Informatics at the Faculty of Automation and Computers, I possess a unique blend of theoretical knowledge and practical experience. My academic background has provided me with a solid understanding of software development principles, algorithms, and data structures.

Upon reviewing the job description, I noticed that the role requires proficiency in at least one back-end programming language. Although I am still in the process of exploring different programming languages, I have gained experience with Python through personal projects, including those hosted on my GitHub profil